In [43]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [44]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Butaritari,KI,2022-07-13 17:17:16,3.0707,172.7902,82.36,78,88,18.86
1,1,Bredasdorp,ZA,2022-07-13 17:17:16,-34.5322,20.0403,58.42,64,43,18.72
2,2,Kavaratti,IN,2022-07-13 17:17:17,10.5669,72.6420,81.36,83,99,19.15
3,3,Vaini,TO,2022-07-13 17:17:17,-21.2000,-175.2000,75.36,94,75,12.66
4,4,Ushuaia,AR,2022-07-13 17:17:17,-54.8000,-68.3000,38.82,75,75,10.36


In [45]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [46]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [47]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [49]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [52]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
9,9,Bambous Virieux,MU,2022-07-13 17:17:19,-20.3428,57.7575,70.05,68,40,11.50
12,12,Sovetskiy,RU,2022-07-13 17:17:20,61.3614,63.5842,71.31,69,55,4.09
27,27,Rikitea,PF,2022-07-13 17:17:24,-23.1203,-134.9692,71.28,83,59,3.33
46,46,Avarua,CK,2022-07-13 17:18:30,-21.2078,-159.7750,71.65,68,75,6.91
59,59,Souillac,MU,2022-07-13 17:18:33,-20.5167,57.5167,70.18,68,40,11.50
76,76,Melfi,IT,2022-07-13 17:18:37,40.9965,15.6514,70.50,25,0,12.75
87,87,Chekhov,RU,2022-07-13 17:18:40,55.1431,37.4711,71.55,66,81,4.12
119,119,Mahebourg,MU,2022-07-13 17:19:49,-20.4081,57.7000,70.09,68,40,11.50
141,141,Yambio,SS,2022-07-13 17:20:56,4.5721,28.3955,70.72,91,95,1.99
178,178,Obo,CF,2022-07-13 17:21:07,5.3998,26.4869,71.82,86,100,2.01


In [53]:
preferred_cities_df.count()

City_ID       26
City          26
Country       26
Date          26
Lat           26
Lng           26
Max Temp      26
Humidity      26
Cloudiness    26
Wind Speed    26
dtype: int64

In [54]:
#Continue with lesson 6.5.4

In [55]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Bambous Virieux,MU,70.05,-20.3428,57.7575,
12,Sovetskiy,RU,71.31,61.3614,63.5842,
27,Rikitea,PF,71.28,-23.1203,-134.9692,
46,Avarua,CK,71.65,-21.2078,-159.7750,
59,Souillac,MU,70.18,-20.5167,57.5167,
76,Melfi,IT,70.50,40.9965,15.6514,
87,Chekhov,RU,71.55,55.1431,37.4711,
119,Mahebourg,MU,70.09,-20.4081,57.7000,
141,Yambio,SS,70.72,4.5721,28.3955,
178,Obo,CF,71.82,5.3998,26.4869,


In [56]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# Iterate through the DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.


In [57]:
hotel_df 

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Bambous Virieux,MU,70.05,-20.3428,57.7575,Casa Tia Villa
12,Sovetskiy,RU,71.31,61.3614,63.5842,Apartment on Kievskaya 29
27,Rikitea,PF,71.28,-23.1203,-134.9692,People ThankYou
46,Avarua,CK,71.65,-21.2078,-159.7750,Paradise Inn
59,Souillac,MU,70.18,-20.5167,57.5167,Shanti Maurice Resort & Spa
76,Melfi,IT,70.50,40.9965,15.6514,HOTEL TERME DI RAPOLLA
87,Chekhov,RU,71.55,55.1431,37.4711,Domik v derevne
119,Mahebourg,MU,70.09,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
141,Yambio,SS,70.72,4.5721,28.3955,Naivasha Hotel
178,Obo,CF,71.82,5.3998,26.4869,


In [58]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [59]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [60]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))